In [ ]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

In [2]:
import google.generativeai as genai
from IPython.display import Markdown

In [3]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key = api_key)

In [ ]:
for model in genai.list_models():
  print(model)

In [8]:
# Documents to embed

DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
#Creating Embedding DataBase with ChromaDB
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

class GeminiEmbeddingFunction(EmbeddingFunction):
  document_mode = True

  def __call__(self, input: Documents) -> Embeddings:
    if self.document_mode:
      embedding_task = "retrieval_document"
    else:
      embedding_task = "retrieval_query"

    retry_policy = {"retry": retry.Retry(predicate= retry.if_transient_error)}

    response = genai.embed_content(
        model = "models/text-embedding-004",
        content = input,
        task_type = embedding_task,
        request_options = retry_policy,
    )
    return response["embedding"]


# -----Embedding --------#
import chromadb
DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name = DB_NAME, embedding_function= embed_fn)
db.add(documents = documents, ids = [str(i) for i in range(len(documents))])


In [11]:
db.count()

3

In [17]:
# Length of the vector
len(db.peek(1)["embeddings"][0])

768

#Retrieval: Find Relevant Documents

In [18]:
# Switch to query mode
embed_fn.document_mode = False

# Searching the chromadb by the relevent query
query = "How to use touchscreen display?"

result = db.query(query_texts = [query], n_results = 1)
[[passage]] = result["documents"]
Markdown(passage)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

#Augmented Generation

In [19]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")
# Prompt to the model
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How to use touchscreen display?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



In [20]:
#Passing it to the model
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer= model.generate_content(prompt)
Markdown(answer.text)

Using your Googlecar's touchscreen is easy!  It's a large screen showing lots of features like maps, music, and climate controls. To use it, just tap the picture (icon) you want. For instance, tap the "Navigation" icon to get directions, or tap the "Music" icon to start playing your tunes.
